In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 1. Indexing Phase

## 파일 불러오기 및 Document 생성

In [2]:
# Document 생성
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('./DE68_04586A_09_QG_Convection_Oven_KO_230403.pdf')
documents = loader.load()

# 개행 문자 제거
for doc in documents:
    doc.page_content = doc.page_content.replace('\n', ' ')
    doc.page_content = doc.page_content.replace('\t', ' ')
    doc.page_content = doc.page_content.replace('/', ' ')

documents

[Document(metadata={'source': './DE68_04586A_09_QG_Convection_Oven_KO_230403.pdf', 'page': 0}, page_content='제품의 외관 및 규격은 제품 개선을 위해 예고 없이 변경될 수 있습니다. 이 제품은 대한민국에서만 사용하도록 만들었으므로 외국에서 사용할 경우 그 품질을 보증하지 않습니다.   (FOR KOREA UNIT STANDARD ONLY) 삼성 스마트 오븐 SmartThings 연결하기 (WiFi 모델에 한함) 1 QR 코드 리더 앱을  실행시켜 제품에  부착된 QR 이미지를  스캔하세요. 2 SmartThings 앱과  제품을 연결해서  사용하세요. 제품 등록   매뉴얼 보기   고객지원  1 QR 코드 리더 앱을  실행시켜 제품에  부착된 QR 이미지를  스캔하세요. 2 제품등록, 매뉴얼 보기,  고객지원 중 원하는  메뉴를 선택하세요.  매뉴얼 보기를 선택하면  구입하신 제품의  설명서를 보실 수  있습니다. 고장 접수 및 재설치 문의 전국 어디서나 1588-3366 (통화요금 : 발신자 부담) 서비스센터 홈페이지 www.samsungsvc.co.kr 모델명, 고장상태, 연락처를 알려주시면 고객 여러분의  궁금한 점을 친절하게 상담, 빠른 서비스를 제공합니다. 고객이 직접 원하는 날짜와 시간에 서비스 접수와  실시간 사이버 상담이 가능한 사이버 서비스센터를  운영하고 있습니다. 고객 상담실    02-541-3000   080-022-3000 인터넷 홈페이지    www.samsung.com sec  • 지구 살리기를 위한 작은 실천으로 종이 매뉴얼을 줄이고, 스마트 폰이나 홈페이지에서 확인할 수  있는 매뉴얼을 제공합니다.   • 제품 사용 설명은 QR 코드로 제공하는 매뉴얼 또는 삼성전자 홈페이지   (http: www.samsung.com sec )를 참조하세요. 한국서비스품질 우수기업이란?  소비자의 의견이나 불만 또는 피해를 신속하고 정확히 처리하는 기업에 대하

## Text Split

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20
)
docs = splitter.split_documents(documents)
print(len(docs))
docs


199


[Document(metadata={'source': './DE68_04586A_09_QG_Convection_Oven_KO_230403.pdf', 'page': 0}, page_content='제품의 외관 및 규격은 제품 개선을 위해 예고 없이 변경될 수 있습니다. 이 제품은 대한민국에서만 사용하도록 만들었으므로 외국에서 사용할 경우 그 품질을 보증하지 않습니다.'),
 Document(metadata={'source': './DE68_04586A_09_QG_Convection_Oven_KO_230403.pdf', 'page': 0}, page_content='그 품질을 보증하지 않습니다.   (FOR KOREA UNIT STANDARD ONLY) 삼성 스마트 오븐 SmartThings 연결하기 (WiFi 모델에 한함) 1 QR 코드'),
 Document(metadata={'source': './DE68_04586A_09_QG_Convection_Oven_KO_230403.pdf', 'page': 0}, page_content='모델에 한함) 1 QR 코드 리더 앱을  실행시켜 제품에  부착된 QR 이미지를  스캔하세요. 2 SmartThings 앱과  제품을 연결해서  사용하세요. 제품 등록   매뉴얼'),
 Document(metadata={'source': './DE68_04586A_09_QG_Convection_Oven_KO_230403.pdf', 'page': 0}, page_content='사용하세요. 제품 등록   매뉴얼 보기   고객지원  1 QR 코드 리더 앱을  실행시켜 제품에  부착된 QR 이미지를  스캔하세요. 2 제품등록, 매뉴얼 보기,  고객지원 중'),
 Document(metadata={'source': './DE68_04586A_09_QG_Convection_Oven_KO_230403.pdf', 'page': 0}, page_content='매뉴얼 보기,  고객지원 중 원하는  메뉴를 선택하세요.  매뉴얼 보기를 선택하면  구

## 임베딩 모델

In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model = 'text-embedding-3-small'   
)


## Vector Store

In [5]:
from langchain_chroma.vectorstores import Chroma
vector_store = Chroma.from_documents(docs, embeddings)

## Retriever를 사용한 유사도 기반

In [9]:
# Retriever를 사용한 검색
query = '오븐의 주요기능에 어떤것들이 있습니까?'

retriever = vector_store.as_retriever(
    search_type = 'similarity',
    search_keywords = {'k' : 3},
)
retrievals = retriever.batch([query])
retrievals


[[Document(metadata={'page': 1, 'source': './DE68_04586A_09_QG_Convection_Oven_KO_230403.pdf'}, page_content='주요 기능 살펴보기 스마트 오븐 사용하기의 상세내용과 형상은 모델에 따라 상이할 수 있습니다. 설치하기 사용 시 알아두기 청소 및 관리하기 사용 가능한 그릇의 종류 간단'),
  Document(metadata={'page': 0, 'source': './DE68_04586A_09_QG_Convection_Oven_KO_230403.pdf'}, page_content='비어 있는 상태에서 작동하지 마세요.  − 본체나 내부의 이상 가열로 인한 고온 상태가 되어 화상의 원인이 됩니다.  − 단, 오븐 기능은 조리실이 비어있는 상태에서 사용'),
  Document(metadata={'page': 1, 'source': './DE68_04586A_09_QG_Convection_Oven_KO_230403.pdf'}, page_content='없애주세요.   ꞏ 오븐 외부를 깨끗이 닦아주세요.  − 조작부는 너무 힘껏 닦지 마세요.  ꞏ 조리 후 회전 접시를 씻어주세요.  − 회전 접시는 중성세제로 깨끗이 씻은 후'),
  Document(metadata={'page': 1, 'source': './DE68_04586A_09_QG_Convection_Oven_KO_230403.pdf'}, page_content='편리하게 사용하기 스마트 오븐 사용하기 생활의 발견 Tip! 홈메이드 사과칩 만들기 사과를 깨끗이 세척 후 얇게 잘라 오븐에  전자레인지 모드로 3 ~ 4\xa0분 정도 돌리세요.')]]

# 2. Retrieval and Generation Phase

## 프롬프트 생성
1. 사용자 질의
2. 검색된 문서

In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ('system', '''
당신은 삼성 서비스 센터 직원입니다. 고객의 기분이 상하지 않게 고객에게 상냥하고 친절하게 context 기반으로만 답변해주세요.

Persona :
당신은 전자 제품에 대한 사랑이 넘치며 열정을 가지고 있고 제품에 대한 지식이 많은 서비스 센터 직원입니다. 전자 제품의 다양한 앱과 기능, 제품 설정, 제품 사용시 주의 할 사항, 제품의 규격에 대한 폭넓은 이해를 가지고 있습니다. 당신의 태도는 친근하고 다가가기 쉬우며, 전자제품의 초보자부터 전문가까지 고객 모두가 쉽게 이해할 수 있도록 설명합니다.

Role:
서비스 센터 직원으로서의 역할은 전자제품의 다양한 기능들을 고객님들이 쉽게 이해 할 수 있도록 설명하고 더 나은 서비스를 제공하는 것입니다. 고객들에게 새로운 기능의 탐험을 이끌고, 다양한 앱과 기능들을 사용할 수 있도록 도와줍니다. 당신의 목표는 고객님들이 전자제품의 뛰어난 앱과 기능들을 쉽고 편리하게 이용할 수 있도록 설명하는 것입니다.

Examples:
- 오븐의 청소 방법을 요청할 경우: 
1. 오븐 내부를 깨끗이 닦아주세요.
    − 조리 후 벽이나 천정에 묻은 찌꺼기는 중성세제를 묻혀 행주로 깨끗이 닦아주세요.
2. 투시창을 깨끗이 닦아주세요.
    − 투시창의 바깥쪽이나 안쪽은 중성세제와 젖은 행주로 닦은 후 마른 헝겊으로 물기를 완전히 없애주세요. 
3. 오븐 외부를 깨끗이 닦아주세요.
    − 조작부는 너무 힘껏 닦지 마세요.
4. 조리 후 회전 접시를 씻어주세요.
    − 회전 접시는 중성세제로 깨끗이 씻은 후 물기를 닦아내고 조리실에 넣어주세요.
5. 보호 커버
    − 제품 내 부품을 보호하는 커버입니다. 임의로 보호 커버를 제거하면 제품 기능에 문제가 발생할 수 있습니다.
    − 보호 커버가 음식물 찌꺼기에 오염되어 있으면 화재의 원인이 되오니 음식물 찌꺼기를 반드시 제거해 주세요.
    − 청소 시 금속 수세미로 닦으면 코팅면이 손상될 수 있으니, 중성세제를 묻혀 행주로 깨끗이 닦아주세요.
    
    '''),
    ('user', '''
    사용자의 질문에 context 만을 이용해 한국어로 답변해 주세요.
    
    question : {query}
    
    context : {context}
    ''')
])
prompt


ChatPromptTemplate(input_variables=['context', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\n당신은 삼성 서비스 센터 직원입니다. 고객의 기분이 상하지 않게 고객에게 상냥하고 친절하게 context 기반으로만 답변해주세요.\n\nPersona :\n당신은 전자 제품에 대한 사랑이 넘치며 열정을 가지고 있고 제품에 대한 지식이 많은 서비스 센터 직원입니다. 전자 제품의 다양한 앱과 기능, 제품 설정, 제품 사용시 주의 할 사항, 제품의 규격에 대한 폭넓은 이해를 가지고 있습니다. 당신의 태도는 친근하고 다가가기 쉬우며, 전자제품의 초보자부터 전문가까지 고객 모두가 쉽게 이해할 수 있도록 설명합니다.\n\nRole:\n서비스 센터 직원으로서의 역할은 전자제품의 다양한 기능들을 고객님들이 쉽게 이해 할 수 있도록 설명하고 더 나은 서비스를 제공하는 것입니다. 고객들에게 새로운 기능의 탐험을 이끌고, 다양한 앱과 기능들을 사용할 수 있도록 도와줍니다. 당신의 목표는 고객님들이 전자제품의 뛰어난 앱과 기능들을 쉽고 편리하게 이용할 수 있도록 설명하는 것입니다.\n\nExamples:\n- 오븐의 청소 방법을 요청할 경우: \n1. 오븐 내부를 깨끗이 닦아주세요.\n    − 조리 후 벽이나 천정에 묻은 찌꺼기는 중성세제를 묻혀 행주로 깨끗이 닦아주세요.\n2. 투시창을 깨끗이 닦아주세요.\n    − 투시창의 바깥쪽이나 안쪽은 중성세제와 젖은 행주로 닦은 후 마른 헝겊으로 물기를 완전히 없애주세요. \n3. 오븐 외부를 깨끗이 닦아주세요.\n    − 조작부는 너무 힘껏 닦지 마세요.\n4. 조리 후 회전 접시를 씻어주세요.\n    − 회전 접시는 중성세제로 깨끗

In [11]:
prompt.invoke({'query' : query, 'context' : retrievals})


ChatPromptValue(messages=[SystemMessage(content='\n당신은 삼성 서비스 센터 직원입니다. 고객의 기분이 상하지 않게 고객에게 상냥하고 친절하게 context 기반으로만 답변해주세요.\n\nPersona :\n당신은 전자 제품에 대한 사랑이 넘치며 열정을 가지고 있고 제품에 대한 지식이 많은 서비스 센터 직원입니다. 전자 제품의 다양한 앱과 기능, 제품 설정, 제품 사용시 주의 할 사항, 제품의 규격에 대한 폭넓은 이해를 가지고 있습니다. 당신의 태도는 친근하고 다가가기 쉬우며, 전자제품의 초보자부터 전문가까지 고객 모두가 쉽게 이해할 수 있도록 설명합니다.\n\nRole:\n서비스 센터 직원으로서의 역할은 전자제품의 다양한 기능들을 고객님들이 쉽게 이해 할 수 있도록 설명하고 더 나은 서비스를 제공하는 것입니다. 고객들에게 새로운 기능의 탐험을 이끌고, 다양한 앱과 기능들을 사용할 수 있도록 도와줍니다. 당신의 목표는 고객님들이 전자제품의 뛰어난 앱과 기능들을 쉽고 편리하게 이용할 수 있도록 설명하는 것입니다.\n\nExamples:\n- 오븐의 청소 방법을 요청할 경우: \n1. 오븐 내부를 깨끗이 닦아주세요.\n    − 조리 후 벽이나 천정에 묻은 찌꺼기는 중성세제를 묻혀 행주로 깨끗이 닦아주세요.\n2. 투시창을 깨끗이 닦아주세요.\n    − 투시창의 바깥쪽이나 안쪽은 중성세제와 젖은 행주로 닦은 후 마른 헝겊으로 물기를 완전히 없애주세요. \n3. 오븐 외부를 깨끗이 닦아주세요.\n    − 조작부는 너무 힘껏 닦지 마세요.\n4. 조리 후 회전 접시를 씻어주세요.\n    − 회전 접시는 중성세제로 깨끗이 씻은 후 물기를 닦아내고 조리실에 넣어주세요.\n5. 보호 커버\n    − 제품 내 부품을 보호하는 커버입니다. 임의로 보호 커버를 제거하면 제품 기능에 문제가 발생할 수 있습니다.\n    − 보호 커버가 음식물 찌꺼기에 오염되어 있으면 화재의 원인이 되오니 음식물 찌꺼기를 반드시 제거해 주세요.\n  

## 모델 생성

In [12]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model = 'gpt-4o',
    temperature = 0.2
)


## Chain 생성

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough : invoke()에 전달된 인자를 받아서 그대로 출력
smart_oven = {'query' : RunnablePassthrough(), 'context' : retriever} | prompt | model | StrOutputParser()


In [17]:
# query = '오븐의 주요기능에 어떤것들이 있습니까?'
smart_oven.invoke(query)


'오븐의 주요 기능에는 스마트 오븐 사용과 관련된 다양한 기능들이 포함되어 있습니다. 모델에 따라 상세 내용과 형상이 다를 수 있지만, 일반적으로 편리한 사용을 위한 다양한 기능들이 제공됩니다. 예를 들어, 전자레인지 모드를 사용하여 간단한 요리를 할 수 있으며, 홈메이드 사과칩과 같은 간단한 요리도 가능합니다. 이러한 기능들은 사용자의 생활을 더욱 편리하게 만들어 줍니다. 추가적으로, 사용 시 주의사항과 청소 및 관리 방법도 함께 제공되어 안전하고 효율적인 사용이 가능합니다.'

In [15]:
smart_oven.invoke('오븐으로 감자칩을 만들 수 있나요?')

NameError: name 'smart_oven' is not defined